# 基礎班時問 1330

問1. 怎麼用python抓share point的資料

程式相關問題通常把問題翻譯成英文的簡短句子丟到 google 裡, 第一筆 stack overflow 的資料有 5, 6 成是解答

https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site

# 影像班時問 1430

In [ ]:
問1.  怎麼使用 GPU 加速 YOLO


- 1. 查 torch 版本
- 2. 安裝相對應的 CUDA toolkit
- 3. 重新安裝 torch 與 cuda 相關的套件

https://developer.nvidia.com/cuda-toolkit-archive/
https://pytorch.org/get-started/previous-versions/

cuda 12.1
pytorch 2.2.2
會使用以下設定
conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia


In [3]:
import torch

In [22]:
# 確認 cuda 可以用
torch.cuda.is_available()

True

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('yolov8n.pt')

In [3]:
model = model.to('cuda') # 這裡可以指定 使用 cuda

## yolo 模型使用

In [23]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model = model.to('cuda') # 這裡可以指定 使用 cuda


In [25]:
results = model("kaohsiung-traffic.jpg")


image 1/1 C:\Users\Charlie Lee\workspace\2024\2024-05-29\kaohsiung-traffic.jpg: 480x640 2 persons, 9 cars, 1 motorcycle, 1 bus, 2 trucks, 94.5ms
Speed: 3.0ms preprocess, 94.5ms inference, 711.5ms postprocess per image at shape (1, 3, 480, 640)


In [26]:
results[0].show()

## yolo 模型訓練步驟
- 準備資料
- 選擇模型
- 訓練
- 使用模型


In [27]:
# 選擇模型
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [28]:
import os
from bs4 import BeautifulSoup

def convert_voc_to_yolo(voc_dir, yolo_dir, classes):
    # 創建 YOLO 目錄（如果不存在）
    if not os.path.exists(yolo_dir):
        os.makedirs(yolo_dir)

    for filename in os.listdir(voc_dir):
        if not filename.endswith('.xml'):
            continue

        # 解析 XML 文件
        xml_path = os.path.join(voc_dir, filename)
        with open(xml_path) as f:
            data = f.read()
        soup = BeautifulSoup(data)

        # 獲取圖片尺寸
        width = int(soup.find('size').find('width').text)
        height = int(soup.find('size').find('height').text)

        # 打開 YOLO 格式的標註文件
        yolo_path = os.path.join(yolo_dir, filename.replace('.xml', '.txt'))
        with open(yolo_path, 'w') as f:
            for obj in soup.find_all('object'):
                # 獲取物件類別
                class_name = obj.find('name').text
                if class_name not in classes:
                    continue
                class_id = classes.index(class_name)

                # 獲取邊界框座標
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                # 轉換為 YOLO 格式
                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                box_width = (xmax - xmin) / width
                box_height = (ymax - ymin) / height

                # 寫入 YOLO 格式
                f.write(f'{class_id} {x_center} {y_center} {box_width} {box_height}\n')

In [30]:
# 定義類別列表（根據你的 Pascal VOC 資料集）
classes = ["with_mask", "without_mask", "mask_weared_incorrect"]  # 替換為你的類別名稱

# 設定 Pascal VOC 和 YOLO 目錄
voc_directory = 'datasets/annotations'
yolo_directory = 'datasets/yolo'

# 執行轉換
convert_voc_to_yolo(voc_directory, yolo_directory, classes)

In [10]:
model = YOLO('yolov8n.pt')
model.train(data='mask.yaml', epochs=1)

New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mask.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\train.cache... 653 images, 0 backgrounds, 0 cor
val: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\valid.cache... 100 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.98G      1.765      2.803      1.389         80        640: 100%|██████████| 41/41 [00:06<00:00,  5.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.335      0.247      0.265      0.181

1 epochs completed in 0.003 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.335      0.247      0.265      0.181
                  Mask        100        284      0.851      0.562      0.694      0.476
                NoMask        100         94      0.129      0.106     0.0939      0.064
             Incorrect        100         14     0.0241     0.0714    0.00582    0.00325
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016102051030>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [11]:
model = YOLO('yolov8n.pt')
model.train(data='mask.yaml', epochs=5)

New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mask.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\train.cache... 653 images, 0 backgrounds, 0 cor
val: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\valid.cache... 100 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      2.97G      1.765      2.803      1.389         80        640: 100%|██████████| 41/41 [00:07<00:00,  5.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<0

                   all        100        392      0.335      0.247      0.265      0.181


...

                   all        100        392      0.852      0.625      0.662      0.448



5 epochs completed in 0.010 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.853      0.626      0.662      0.449
                  Mask        100        284      0.784      0.948      0.943      0.652
                NoMask        100         94      0.773      0.809      0.812      0.531
             Incorrect        100         14          1       0.12      0.229      0.165
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001608B18A710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [12]:
model = YOLO('yolov8n.pt')
model.train(data='mask.yaml', epochs=10)

New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mask.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\train.cache... 653 images, 0 backgrounds, 0 cor
val: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\valid.cache... 100 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train3
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      2.41G      1.768      2.938      1.432         59        640: 100%|██████████| 41/41 [00:07<00:00,  5.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<0

                   all        100        392    0.00819      0.486       0.16      0.113


...

                   all        100        392      0.789      0.681      0.742        0.5



10 epochs completed in 0.025 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.839      0.694      0.728      0.506
                  Mask        100        284      0.888      0.915      0.945      0.666
                NoMask        100         94      0.827      0.809      0.829      0.527
             Incorrect        100         14      0.802      0.357      0.412      0.327
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000160F4B0F370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [13]:
model = YOLO('yolov8n.pt')
model.train(data='mask.yaml', epochs=100)

New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mask.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\train.cache... 653 images, 0 backgrounds, 0 cor
val: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\valid.cache... 100 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         3G      1.765      2.803      1.389         80        640: 100%|██████████| 41/41 [00:47<00:00,  1.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<0

                   all        100        392      0.335      0.247      0.265      0.181


...

                   all        100        392      0.926      0.783      0.872      0.621



100 epochs completed in 0.215 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.931      0.827      0.897      0.633
                  Mask        100        284      0.926      0.915      0.964      0.691
                NoMask        100         94      0.924      0.779      0.887      0.604
             Incorrect        100         14      0.941      0.786       0.84      0.602
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016190AE5000>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [14]:
model = YOLO('yolov8n.pt')
model.train(data='mask.yaml', epochs=1000)

New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mask.yaml, epochs=1000, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

train: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\train.cache... 653 images, 0 backgrounds, 0 cor
val: Scanning C:\Users\Charlie Lee\workspace\2024\2024-05-29\datasets\valid.cache... 100 images, 0 backgrounds, 0 corru


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train5
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/1000       3.1G      1.821      3.356      1.501         80        640: 100%|██████████| 41/41 [00:51<00:00,  1.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<0

                   all        100        392    0.00816      0.333      0.172     0.0998


...

                   all        100        392      0.923      0.754      0.864      0.597
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 186, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



286 epochs completed in 0.569 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 6.2MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.2.24  Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0


                   all        100        392      0.914      0.847      0.881      0.632
                  Mask        100        284      0.919      0.923      0.965      0.688
                NoMask        100         94      0.907      0.833      0.888      0.603
             Incorrect        100         14      0.916      0.786       0.79      0.605
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016190EC4BE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [47]:
trained_model = YOLO('runs/detect/train42/weights/best.pt')

In [ ]:
results = trained_model('Masks_faces_courtesy-of-Ashley-Ruba.jpg')

In [60]:
results[0].show()

# 綜合時間 1530

In [62]:
import pandas as pd


In [83]:
last_name = 'Todd'
all_customers = pd.read_csv('customers.csv')[['customer_id', 'first_name', 'last_name']]
customers = all_customers[all_customers['last_name'] == last_name]

## https://raw.githubusercontent.com/pochangl/2024/main/2024-05-08/orders.csv
orders = pd.read_csv('orders.csv')[['order_id', 'customer_id', 'order_date']]

filtered_orders = pd.merge(left=customers, right=orders, on='customer_id')


In [84]:
groups = filtered_orders.groupby('customer_id').size()

In [86]:
type(groups)

pandas.core.series.Series

In [87]:
groups = groups.reset_index()
type(groups)

pandas.core.frame.DataFrame

In [89]:
groups.columns = ['customer_id', 'count']

In [90]:
groups

,customer_id,count
0,2,3
1,440,1


In [93]:
order_count = pd.merge(left=groups, right=all_customers, on='customer_id')
order_count = order_count[['last_name','first_name', 'count']]
order_count['name'] = order_count['first_name'] + ' ' + order_count['last_name']

In [92]:
order_count

,last_name,first_name,count,name
0,Todd,Kasha,3,Kasha Todd
1,Todd,Yevette,1,Yevette Todd
